In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/fastai_full'
%reload_ext autoreload
%autoreload 2

Mounted at /content/gdrive


In [2]:
import pandas as pd              
import numpy as np         
import os                  
from random import shuffle 
import scipy
import skimage
from skimage.transform import resize
import cv2
import time
import matplotlib.pyplot as plt

In [3]:
train_labels = pd.read_csv(base_dir+'/stage_2_train_labels.csv')
df = train_labels.drop(['x','y','width','height'],axis='columns')

In [1]:
def get_data(Dir, df=df):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    i=0
    for nextdir in os.listdir(Dir):
        if nextdir.endswith('.jpg'):
            i+=1
            nextdir, label = nextdir, int(df[df.patientId==nextdir[:-4]].Target.array[0])
            image = cv2.imread(Dir +'/'+nextdir, cv2.IMREAD_GRAYSCALE)
            image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
            image = skimage.transform.resize(image, (299, 299, 3))
                        #img_file = scipy.misc.imresize(arr=img_file, size=(299, 299, 3))
            image= np.asarray(image)
            if i%5==0:
                X_test.append(image)
                y_test.append(label)
            else:
                X_train.append(image)
                y_train.append(label)
        if i>3000:
            break
                    
    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    X_test = np.asarray(X_test)
    y_test = np.asarray(y_test)
    return X_train, y_train, X_test, y_test

NameError: ignored

In [ ]:
X_train, y_train, X_test, y_test = get_data(base_dir+'/grey_train')

In [ ]:
X_test.shape

(300, 299, 299, 3)

In [ ]:
X_train = X_train.reshape(1201, 3, 299, 299)
X_test = X_test.reshape(300, 3, 299, 299)

In [ ]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint , LearningRateScheduler
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, epsilon=0.0001, patience=1, verbose=1)

In [ ]:
filepath="transferlearning_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
from keras.models import Sequential , Model
from keras.layers import Dense , Activation
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop , Adadelta , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_data_format('channels_last')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
from keras.applications.inception_v3 import InceptionV3
# create the base pre-trained model
base_model = InceptionV3()

96124928/96112376 [==============================] - 1s 0us/step


In [ ]:
x = base_model.output
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(2, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

In [ ]:
batch_size = 64
epochs = 5

In [ ]:
def swish_activation(x):
    return (K.sigmoid(x) * x)

model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=(3,299,299)))
model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(3,299,299)))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))

model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))

model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))

model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))

model.add(Flatten())

model.add(Dense(64, activation=swish_activation))
model.add(Dropout(0.4))
model.add(Dense(2 , activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=0.00005),
                  metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test , y_test) ,callbacks=[lr_reduce,checkpoint] ,
          epochs=epochs)

Epoch 1/5
38/38 [==============================] - 55s 991ms/step - loss: 0.6033 - accuracy: 0.8326 - val_loss: 0.4060 - val_accuracy: 0.8600

Epoch 00001: val_accuracy improved from -inf to 0.86000, saving model to transferlearning_weights.hdf5
Epoch 2/5
38/38 [==============================] - 35s 912ms/step - loss: 0.4231 - accuracy: 0.8569 - val_loss: 0.4046 - val_accuracy: 0.8600

Epoch 00002: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00002: val_accuracy did not improve from 0.86000
Epoch 3/5
38/38 [==============================] - 35s 915ms/step - loss: 0.4375 - accuracy: 0.8412 - val_loss: 0.4029 - val_accuracy: 0.8600

Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-07.

Epoch 00003: val_accuracy did not improve from 0.86000
Epoch 4/5
38/38 [==============================] - 34s 891ms/step - loss: 0.3956 - accuracy: 0.8662 - val_loss: 0.4029 - val_accuracy: 0.8600

Epoch 00004: ReduceLROnPlateau reducing learning rate